# ETL_EDA    
This file records the process of acquiring raw data, traforming them, and loading them into a MongoDB. The data are store (almost) in their raw form.

In [5]:
from get_data import *

## 1. Raw html data from basketball-reference.com containing NBA data
The data is embeded in a plain html website page and thus we need to crawl the website and find the specific tags to retrieve the tablular data that we need.

The website page source is at https://www.basketball-reference.com/leagues/NBA_2020_totals.html

It contains the overall performance of each officially registered NBA player in season 2019-2020.

At the time this project is being contructed, the 2019-2020 NBA gaming season is not over yet. Thus, the dataset will be updated minutely when there is a game playing. Also, there will be no update on the dataset if there is no game playing on the day.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pds

url = 'https://www.basketball-reference.com/leagues/NBA_2020_totals.html'
html_file = requests.get(url)
soup = BeautifulSoup(html_file.text)

# Demonstration of the raw data table
table_body = soup.find('tbody')  # find the table
rows = table_body.find_all('tr') # find each row in the table

Display the first 10 players' raw information

In [9]:
i = 0
for r in rows:
    print(r)
    if i == 10:
        break
    i+=1

<tr class="full_table"><th class="right " csk="1" data-stat="ranker" scope="row">1</th><td class="left " csk="Adams,Steven" data-append-csv="adamsst01" data-stat="player"><a href="/players/a/adamsst01.html">Steven Adams</a></td><td class="center " data-stat="pos">C</td><td class="right " data-stat="age">26</td><td class="left " data-stat="team_id"><a href="/teams/OKC/2020.html">OKC</a></td><td class="right " data-stat="g">22</td><td class="right " data-stat="gs">22</td><td class="right " data-stat="mp">600</td><td class="right " data-stat="fg">106</td><td class="right " data-stat="fga">174</td><td class="right " data-stat="fg_pct">.609</td><td class="right iz" data-stat="fg3">0</td><td class="right " data-stat="fg3a">1</td><td class="right non_qual" data-stat="fg3_pct">.000</td><td class="right " data-stat="fg2">106</td><td class="right " data-stat="fg2a">173</td><td class="right " data-stat="fg2_pct">.613</td><td class="right " data-stat="efg_pct">.609</td><td class="right " data-stat

## 2. Raw Data to Documents/Dicts      
The table text is found and will need to be extracted from the html string. The missing values indicates that the player either does not play a game yet or it is not applicable for him. Thus, all the missing values will be set to 0. The data will first be stored as a list of dictionary containing each player's record.

In [6]:
percents = ['FG%', '3P%', '2P%', 'eFG%', 'FT%']
data = [] # a list of records.

# get column names
feature_names = [row.text.strip() for row in soup.find('thead').find_all("th")]
rank = 1
for r in rows:
    cols = r.find_all('td')
    if len(cols):
        cols = [element.text.strip() for element in cols]
        cols = [str(rank)] + [element if element else '0' for element in cols]   
        rank += 1
        record = dict() # store the record for this player
        for i in range(len(feature_names)):
            name = feature_names[i]
            if name in percents:
                record[name] = float(cols[i])
            else:
                try:
                    record[name] = int(cols[i])
                except:
                    record[name] = cols[i]
        data.append(record)

Displaying the first record

In [7]:
data[0]

{'Rk': 1,
 'Player': 'Steven Adams',
 'Pos': 'C',
 'Age': 26,
 'Tm': 'OKC',
 'G': 23,
 'GS': 23,
 'MP': 629,
 'FG': 110,
 'FGA': 182,
 'FG%': 0.604,
 '3P': 0,
 '3PA': 1,
 '3P%': 0.0,
 '2P': 110,
 '2PA': 181,
 '2P%': 0.608,
 'eFG%': 0.604,
 'FT': 35,
 'FTA': 68,
 'FT%': 0.515,
 'ORB': 77,
 'DRB': 144,
 'TRB': 221,
 'AST': 63,
 'STL': 10,
 'BLK': 30,
 'TOV': 35,
 'PF': 44,
 'PTS': 255}

The data can also be displayed as a pandas. Let's show the first few rows in a tabular format.

In [16]:
import pandas as pd
df = pd.DataFrame.from_records(data)
df.head()

,2P,2P%,2PA,3P,3P%,3PA,AST,Age,BLK,DRB,...,PF,PTS,Player,Pos,Rk,STL,TOV,TRB,Tm,eFG%
0,106,.613,173,0,.000,1,62,26,29,139,...,43,246,Steven Adams,C,1,10,31,210,OKC,.609
1,148,.599,247,1,.125,8,120,22,31,209,...,70,396,Bam Adebayo,C,2,36,75,276,MIA,.586
2,161,.519,310,13,.342,38,55,34,41,117,...,51,432,LaMarcus Aldridge,C,3,14,37,164,SAS,.519
3,22,.319,69,27,.342,79,40,21,2,43,...,27,136,Nickeil Alexander-Walker,SG,4,9,24,47,NOP,.422
4,23,.469,49,15,.349,43,19,24,1,36,...,21,106,Grayson Allen,SG,5,5,15,38,MEM,.495


## 3. Upsert MongoDB         
Here, the data can is a list of dicts which is what we want for the MongoDB. We will upsert each record to MongoDB. Due to the fact that 1) there is no time log for each record and 2) the data set is not huge and 3) the information of each player may change minutely, all the records will be updated each time the new data is fetched.

In [ ]:
import pymongo
client = pymongo.MongoClient()

db = client.get_database("nba")
collection = db.get_collection("energy")
for record in data:
    collection.replace_one(
        filter={'Player': record['Player']},    # locate the document is exists
        replacement=record,                     # latest document
        upsert = True)                          # update if exists, insert if not